<a href="https://colab.research.google.com/github/hyomin14/ML_TeamProject/blob/main/imbalanced_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/MachineLearning/heart_2020_cleaned.csv')
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301717 entries, 0 to 319794
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      301717 non-null  object 
 1   BMI               301717 non-null  float64
 2   Smoking           301717 non-null  object 
 3   AlcoholDrinking   301717 non-null  object 
 4   Stroke            301717 non-null  object 
 5   PhysicalHealth    301717 non-null  float64
 6   MentalHealth      301717 non-null  float64
 7   DiffWalking       301717 non-null  object 
 8   Sex               301717 non-null  object 
 9   AgeCategory       301717 non-null  object 
 10  Race              301717 non-null  object 
 11  Diabetic          301717 non-null  object 
 12  PhysicalActivity  301717 non-null  object 
 13  GenHealth         301717 non-null  object 
 14  SleepTime         301717 non-null  float64
 15  Asthma            301717 non-null  object 
 16  KidneyDisease     30

## 전처리

In [ ]:
categorical_col = ['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'PhysicalActivity', 'Asthma', 'KidneyDisease', 'SkinCancer']
for col in categorical_col:
    df[col] = df[col].map({'Yes': 1, 'No': 0})
df['Sex'] = df['Sex'].map({'Female': 0, 'Male': 1})
df['GenHealth'] = df['GenHealth'].map({'Poor': 0, 'Fair': 1, 'Good' : 2, 'Very good' : 3, 'Excellent' : 4})

df.loc[df['Diabetic']=='Yes (during pregnancy)', 'Diabetic'] = '2'
df.loc[df['Diabetic']=='Yes', 'Diabetic'] = '2'
df.loc[df['Diabetic']=='No, borderline diabetes', 'Diabetic'] = '1'
df.loc[df['Diabetic']=='No', 'Diabetic'] = '0'

df['AgeCategory'] = df['AgeCategory'].map(lambda x:x.split('-'))
df['AgeCategory'] = df['AgeCategory'].map(lambda x : x[0][:2] if len(x)==1 else x[0])
df['AgeCategory'] = df['AgeCategory'].astype('int64')

df = pd.get_dummies(df, columns = ['Race'])


In [ ]:
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,...,SleepTime,Asthma,KidneyDisease,SkinCancer,Race_American Indian/Alaskan Native,Race_Asian,Race_Black,Race_Hispanic,Race_Other,Race_White
0,0,16.60,1,0,0,3.0,30.0,0,0,55,...,5.0,1,0,1,0,0,0,0,0,1
1,0,20.34,0,0,1,0.0,0.0,0,0,80,...,7.0,0,0,0,0,0,0,0,0,1
2,0,26.58,1,0,0,20.0,30.0,0,1,65,...,8.0,1,0,0,0,0,0,0,0,1
3,0,24.21,0,0,0,0.0,0.0,0,0,75,...,6.0,0,0,1,0,0,0,0,0,1
4,0,23.71,0,0,0,28.0,0.0,1,0,40,...,8.0,0,0,0,0,0,0,0,0,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301717 entries, 0 to 319794
Data columns (total 23 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   HeartDisease                         301717 non-null  int64  
 1   BMI                                  301717 non-null  float64
 2   Smoking                              301717 non-null  int64  
 3   AlcoholDrinking                      301717 non-null  int64  
 4   Stroke                               301717 non-null  int64  
 5   PhysicalHealth                       301717 non-null  float64
 6   MentalHealth                         301717 non-null  float64
 7   DiffWalking                          301717 non-null  int64  
 8   Sex                                  301717 non-null  int64  
 9   AgeCategory                          301717 non-null  int64  
 10  Diabetic                             301717 non-null  object 
 11  PhysicalActiv

In [ ]:
from sklearn.preprocessing import StandardScaler
df_scaled = df.copy()
scaler = StandardScaler()
df_scaled[['BMI','PhysicalHealth', 'MentalHealth', 'SleepTime']] = scaler.fit_transform(df_scaled[['BMI','PhysicalHealth', 'MentalHealth', 'SleepTime']])

## Baseline model -> Logistic

In [ ]:
X_features = df_scaled.drop('HeartDisease', axis=1, inplace=False)
y_target = df_scaled['HeartDisease']
print(sum(y_target==0)/y_target.shape[0])
print(sum(y_target==1)/y_target.shape[0])

0.9096471196518592
0.0903528803481408


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.3)
print(y_train.shape, y_test.shape)
print(sum(y_train==0)/y_train.shape[0])
print(sum(y_train==1)/y_train.shape[0])

(211201,) (90516,)
0.9094322470064062
0.09056775299359378


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print(accuracy_score(y_test, lr_preds))
print(f1_score(y_test, lr_preds))
print(roc_auc_score(y_test, lr_preds))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9111096380750364
0.19844590555887626
0.5557151028081527


## oversampling

### Random OverSampling

In [ ]:
# 소수 클래스 데이터 단순 복제, 숫자가 늘어나기 때문에 더 많은 가중치 받는 원리 -> 과적합 위험 존재
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

print(sum(y_target==0)/y_target.shape[0])
print(sum(y_target==1)/y_target.shape[0])

ros = RandomOverSampler()
X_res, y_res = ros.fit_resample(X_features, y_target)

print(Counter(y_target))
print(Counter(y_res))

0.9096471196518592
0.0903528803481408
Counter({0: 274456, 1: 27261})
Counter({0: 274456, 1: 274456})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy: 0.7587172231196182
f1 score: 0.7633658707990638
roc auc score: 0.7586949159972814


### SMOTE

In [ ]:
# 소수 클래스에 속하는 데이터 주변에 원본 데이터와 동일하지 않으면서 소수 클래스에 해당하는 가상의 데이터를 생성(minority class에서 synthetic 샘플을 생성) + knn
# knn으로 가까운 minority class 들을 찾은 후 0과 1사이의 랜덤한 값을 선택한 후 해당 값으로 내분하는 점을 새로운 샘플로 생성하는 기법
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X_features, y_target)

print('Before:', Counter(y_target))
print('After:', Counter(y_smote))

Before: Counter({0: 274456, 1: 27261})
After: Counter({0: 274456, 1: 274456})


In [ ]:
smote.nn_k_

NearestNeighbors(n_neighbors=6)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy: 0.7714089655926254
f1 score: 0.778600542279574
roc auc score: 0.77149022129568


##### SMOTE의 k_neighbors 하이퍼파라미터 조정

In [ ]:
smote = SMOTE(k_neighbors=10)
X_smote, y_smote = smote.fit_resample(X_features, y_target)
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy: 0.7740505483561461
f1 score: 0.7802945309824393
roc auc score: 0.7740181416740515


### BorderlineSMOTE

In [ ]:
# 소수 클래스에 속하는 데이터 하나를 선정해 N개 주변을 탐색한 후 이 N개 데이터 중 다수 클래스에 속하는 데이터가 몇개인지 확인 -> Noise, Danger, Safe 관측치로 나눔
# Danger 관측치에 대해서만 SMOTE 적용
from imblearn.over_sampling import BorderlineSMOTE
bsmote = BorderlineSMOTE()
X_bsmote, y_bsmote = bsmote.fit_resample(X_features, y_target)

print('Before:', Counter(y_target))
print('After:', Counter(y_bsmote))

print(bsmote.sampling_strategy_)
print(bsmote.nn_k_)
print(bsmote.nn_m_)

Before: Counter({0: 274456, 1: 27261})
After: Counter({0: 274456, 1: 274456})
OrderedDict([(1, 247195)])
NearestNeighbors(n_neighbors=6)
NearestNeighbors(n_neighbors=11)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_bsmote, y_bsmote, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy: 0.8021727777305464
f1 score: 0.8096170366950494
roc auc score: 0.8021401147934948


### ADASYN

In [ ]:
# 가중치를 통해 SMOTE 적용, 소수 클래스 주변의 다수 클래스 수에 따라 유동적으로 오버샘플링 할 데이터 개수를 생성
from imblearn.over_sampling import ADASYN
ads = ADASYN()
X_ads, y_ads = ads.fit_resample(X_features, y_target)

print('Before:', Counter(y_target))
print('After:', Counter(y_ads))

print(ads.nn_)

Before: Counter({0: 274456, 1: 27261})
After: Counter({1: 277099, 0: 274456})
NearestNeighbors(n_neighbors=6)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_ads, y_ads, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy: 0.7556491626729197
f1 score: 0.763425508759201
roc auc score: 0.7556118505368855


### SVMSMOTE

In [ ]:
from imblearn.over_sampling import SVMSMOTE 
sm = SVMSMOTE()
X_res, y_res = sm.fit_resample(X_features, y_target)

print('Before:', Counter(y_target))
print('After:', Counter(y_res))
print(sm.nn_k_)
print(sm.nn_m_)
print(sm.svm_estimator)

Before: Counter({0: 274456, 1: 27261})  
After: Counter({0: 274456, 1: 274456})  
NearestNeighbors(n_neighbors=6)  
NearestNeighbors(n_neighbors=11)  
None  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

accuracy: 0.8540571067685245  
f1 score: 0.8549809018663673  
roc auc score: 0.8541038360828026  

### SMOTENC

In [ ]:
from imblearn.over_sampling import SMOTENC
sm = SMOTENC(categorical_features=[1,2,3,6,7,9,10,13,14,15,16,17,18,19,20,21])
X_res, y_res = sm.fit_resample(X_features, y_target)
print(f'Resampled dataset samples per class {Counter(y_res)}')

Resampled dataset samples per class Counter({0: 274456, 1: 274456})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

accuracy: 0.7723805822412767  
f1 score: 0.7793495217071376  
roc auc score: 0.7723053881512035  

https://imbalanced-learn.org/stable/references/over_sampling.html

### MSMOTE

In [ ]:
import smote_variants as sv
oversampler= sv.distance_SMOTE()
X_ms, y_ms = oversampler.sample(X_features, y_target)
print('Before:', Counter(y_target))
print('After:', Counter(y_ms))

2022-05-28 14:40:54,476:INFO:distance_SMOTE: Running sampling via ('distance_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")


KeyError: ignored

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_ms, y_ms, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

## UnderSampling

### Random Under Sampling

In [ ]:
# 높은 비중을 차지하는 클래스의 값들을 임의로 제거 - 유용한 데이터 삭제될 수 있고 편향된 데이터, 모집단 대표 X 등등
from imblearn.under_sampling import RandomUnderSampler 
rus = RandomUnderSampler()
X_res, y_res = rus.fit_resample(X_features, y_target)
print('Before:', Counter(y_target))
print('After:', Counter(y_res))

Before: Counter({0: 274456, 1: 27261})
After: Counter({0: 27261, 1: 27261})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

accuracy: 0.761142018707587
f1 score: 0.7646243749623471
roc auc score: 0.761398964396835


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


### Tomek Links

In [ ]:
# 클래스가 다른 두 데이터가 아주 가까이 붙어있으면 토멕링크
# 다수 클래스에 속하는 데이터를 제외하는 방법으로 경계선을 다수 클래스쪽으로 밀어붙이는 효과
from imblearn.under_sampling import *
tomek = TomekLinks()
# majority, all
X_tomek, y_tomek = tomek.fit_resample(X_features, y_target)

Before: Counter({0: 274456, 1: 27261})
After: Counter({0: 27261, 1: 27261})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tomek, y_tomek, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy: 0.9086332574031891
f1 score: 0.21598905394839718
roc auc score: 0.5614366526364927


### CNN(Condensed Nearest Neighbor)

In [ ]:
# 1-NN 모형으로 분류되지 않는 데이터만 남기는 방법
cnn = CondensedNearestNeighbour()
X_cnn, y_cnn = cnn.fit_resample(X_features, y_target)
print('Before:', Counter(y_target))
print('After:', Counter(y_cnn))
print(cnn.estimator_)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_cnn, y_cnn, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

### OSS(One-side Selection)

In [ ]:
# Tomek links 와 CNN을 같이 수행하는 방식
oss = OneSidedSelection()
X_oss, y_oss = oss.fit_resample(X_features, y_target)
print('Before:', Counter(y_target))
print('After:', Counter(y_oss))
print(oss.estimator_)

Before: Counter({0: 274456, 1: 27261})  
After: Counter({0: 265385, 1: 27261})  
KNeighborsClassifier(n_neighbors=1)  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_oss, y_oss, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

accuracy: 0.9097432626375379  
f1 score: 0.2246575342465754  
roc auc score: 0.564670590913748  

### Edited Nearest Neighbours

In [ ]:
# 다수 클래스 데이터 중 가장 가까운 k(n_neighbors)개의 데이터가 모두(kind_sel="all") 또는 다수(kind_sel="mode") 다수 클래스가 아니면 삭제하는 방법
# 소수 클래스 주변의 다수 클래스 데이터는 사라짐
enn = EditedNearestNeighbours()
X_res, y_res = enn.fit_resample(X_features, y_target)

print('Before:', Counter(y_target))
print('After:', Counter(y_res))
print(enn.nn_)

Before: Counter({0: 274456, 1: 27261})  
After: Counter({0: 225788, 1: 27261})  
NearestNeighbors(n_neighbors=4)  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

accuracy: 0.9102944082197194  
f1 score: 0.4576298184135075  
roc auc score: 0.664545051465775  

### Neighbourhood Cleaning Rule

In [ ]:
# CNN(Condensed Nearest Neighbour)과 ENN(Edited Nearest Neighbours) 같이 수행
ncr = NeighbourhoodCleaningRule()
X_res, y_res = ncr.fit_resample(X_features, y_target)

print('Before:', Counter(y_target))
print('After:', Counter(y_res))
print(ncr.nn_)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

이 외에도 RepeatedEditedNearestNeighbours, AllKNN, InstanceHardnessThreshold 등이 존재함.  
https://imbalanced-learn.org/stable/references/under_sampling.html

## 복합 방식

### SMOTE-Tomek

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from collections import Counter

In [ ]:
from imblearn.combine import SMOTETomek

smote_tomek = SMOTETomek()
X_res, y_res = smote_tomek.fit_resample(X_features, y_target)

print('Before:', Counter(y_target))
print('After:', Counter(y_res))
print(smote_tomek.smote_)
print(smote_tomek.tomek_)

Before: Counter({0: 274456, 1: 27261})
After: Counter({0: 267518, 1: 267518})
SMOTE()
TomekLinks(sampling_strategy='all')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy: 0.7794481375108248
f1 score: 0.7878871399725579
roc auc score: 0.7794478896737344


### SMOTEENN

In [ ]:
from imblearn.combine import SMOTEENN

smoteenn = SMOTEENN()
X_res, y_res = smoteenn.fit_resample(X_features, y_target)

print('Before:', Counter(y_target))
print('After:', Counter(y_res))
print(smoteenn.smote_)
print(smoteenn.enn_)

Before: Counter({0: 274456, 1: 27261})
After: Counter({1: 234789, 0: 201849})
SMOTE()
EditedNearestNeighbours(sampling_strategy='all')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
lr_preds = model.predict(X_test)
print('accuracy:', accuracy_score(y_test, lr_preds))
print('f1 score:', f1_score(y_test, lr_preds))
print('roc auc score:', roc_auc_score(y_test, lr_preds))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy: 0.8479220105044583
f1 score: 0.8601416766710898
roc auc score: 0.8460646743726089


#### SMOTEENN 전 데이터

In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns

pca = PCA(n_components=2)
pca.fit(X_features)
X_pca = pca.transform(X_features)

pca_df = pd.DataFrame(X_pca, columns=['pca_1', 'pca_2'])
pca_df['target'] = y_target

sns.scatterplot(x='pca_1', y='pca_2', hue='target', data=pca_df)

#### SMOTEENN 후 데이터

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_res)
X_pca = pca.transform(X_res)

pca_df = pd.DataFrame(X_pca, columns=['pca_1', 'pca_2'])
pca_df['target'] = y_res

sns.scatterplot(x='pca_1', y='pca_2', hue='target', data=pca_df)